<a href="https://colab.research.google.com/github/Saha3902/Big-data/blob/main/tp_Big_data02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================
# TP02: التعامل مع ملفات CSV ضخمة (حوالي 5GB) في Google Colab
# ============================================================

!pip install dask[complete] pyarrow fastparquet tqdm --quiet

import os
import pandas as pd
import numpy as np
import dask.dataframe as dd
from time import perf_counter
from tqdm import tqdm
import glob

# ============================================================
# 1️⃣ إنشاء أو تحميل ملف CSV كبير (~5GB)
# ============================================================

base_dir = "/content"
csv_path = os.path.join(base_dir, "large_5gb_sample.csv")

# ✳️ لتقليل وقت التجربة في Colab يمكن تقليل عدد الصفوف مؤقتًا (مثلاً 5 مليون بدلاً من 28 مليون)
rows = 5_000_000   # عدّل إلى 28_000_000 للحصول على ~5GB فعلي
cols = 5

if not os.path.exists(csv_path):
    print("🔹 جاري إنشاء ملف CSV كبير... قد يستغرق هذا بضع دقائق ⏳")
    np.random.seed(42)
    df = pd.DataFrame({
        'id': np.arange(rows),
        'fare_amount': np.random.rand(rows) * 100,
        'distance_km': np.random.rand(rows) * 50,
        'passengers': np.random.randint(1, 5, size=rows),
        'payment_type': np.random.choice(['cash', 'card'], size=rows)
    })
    df.to_csv(csv_path, index=False)
    size_gb = os.path.getsize(csv_path) / (1024**3)
    print(f"✅ تم إنشاء الملف: {csv_path} ({size_gb:.2f} GB)")
else:
    print(f"✅ الملف موجود مسبقًا: {csv_path}")


# ============================================================
# 2️⃣ الطريقة الأولى: pandas.read_csv(chunksize)
# ============================================================

print("\n=== الطريقة 1: pandas.read_csv(chunksize) ===")

start = perf_counter()
total_sum = 0
count = 0
chunksize = 500_000  # حجم كل جزء

# ✅ استخدام tqdm لشريط التقدم في Colab
total_chunks = int(np.ceil(rows / chunksize))
for chunk in tqdm(pd.read_csv(csv_path, chunksize=chunksize, usecols=['fare_amount']),
                  total=total_chunks, desc="قراءة الملف بـ Pandas"):
    s = chunk['fare_amount'].dropna()
    total_sum += s.sum()
    count += s.count()

pandas_mean = total_sum / count
end = perf_counter()
pandas_time = end - start
print(f"📊 المتوسط (pandas): {pandas_mean:.4f}")
print(f"⏱️ الزمن: {pandas_time:.2f} ثانية")


# ============================================================
# 3️⃣ الطريقة الثانية: Dask DataFrame
# ============================================================

print("\n=== الطريقة 2: Dask DataFrame ===")

start = perf_counter()
ddf = dd.read_csv(csv_path, assume_missing=True, usecols=['fare_amount'])
dask_mean = ddf['fare_amount'].mean().compute()
end = perf_counter()
dask_time = end - start

print(f" المتوسط (Dask): {dask_mean:.4f}")
print(f" الزمن: {dask_time:.2f} ثانية")


# ============================================================
# 4️⃣ الطريقة الثالثة: التحويل إلى Parquet (ضغط وقراءة أسرع)
# ============================================================

print("\n=== الطريقة 3: تحويل CSV إلى Parquet ===")

out_dir = os.path.join(base_dir, "parquet_dataset")
os.makedirs(out_dir, exist_ok=True)

start = perf_counter()
i = 0
for chunk in tqdm(pd.read_csv(csv_path, chunksize=1_000_000),
                  desc="تحويل إلى Parquet"):
    fn = os.path.join(out_dir, f"part_{i}.parquet")
    chunk.to_parquet(fn, engine='pyarrow', index=False)
    i += 1
end = perf_counter()
parquet_time = end - start

parquet_files = glob.glob(os.path.join(out_dir, "*.parquet"))
total_parquet_size = sum(os.path.getsize(f) for f in parquet_files) / (1024**2)

print(f"✅ تم إنشاء {len(parquet_files)} ملف Parquet")
print(f"📦 إجمالي الحجم: {total_parquet_size:.2f} MB")
print(f"⏱️ زمن التحويل: {parquet_time:.2f} ثانية")


# ============================================================
# 5️⃣ المقارنة النهائية
# ============================================================

print("\n=== 📊 مقارنة النتائج ===")
csv_size = os.path.getsize(csv_path) / (1024**3)
print(f"حجم CSV الأصلي: {csv_size:.2f} GB")
print(f"Pandas: المتوسط {pandas_mean:.4f} - الزمن {pandas_time:.2f}s")
print(f"Dask:   المتوسط {dask_mean:.4f} - الزمن {dask_time:.2f}s")
print(f"Parquet الحجم: {total_parquet_size/1024:.2f} GB - زمن التحويل: {parquet_time:.2f}s")

print("\n✅ انتهى التنفيذ بنجاح.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.3 MB/s eta 0:00:00
🔹 جاري إنشاء ملف CSV كبير... قد يستغرق هذا بضع دقائق ⏳
✅ تم إنشاء الملف: /content/large_5gb_sample.csv (0.24 GB)

=== الطريقة 1: pandas.read_csv(chunksize) ===


قراءة الملف بـ Pandas: 100%|██████████| 10/10 [00:02<00:00,  4.14it/s]


📊 المتوسط (pandas): 49.9987
⏱️ الزمن: 2.43 ثانية

=== الطريقة 2: Dask DataFrame ===
 المتوسط (Dask): 49.9987
 الزمن: 4.30 ثانية

=== الطريقة 3: تحويل CSV إلى Parquet ===


تحويل إلى Parquet: 5it [00:04,  1.10it/s]

✅ تم إنشاء 5 ملف Parquet
📦 إجمالي الحجم: 101.21 MB
⏱️ زمن التحويل: 4.56 ثانية

=== 📊 مقارنة النتائج ===
حجم CSV الأصلي: 0.24 GB
Pandas: المتوسط 49.9987 - الزمن 2.43s
Dask:   المتوسط 49.9987 - الزمن 4.30s
Parquet الحجم: 0.10 GB - زمن التحويل: 4.56s

✅ انتهى التنفيذ بنجاح.
